In [1]:
import vllm
import rich
import rich.panel
import rich.markup


/home/mila/g/gagnonju/.mambaforge/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/mila/g/gagnonju/.mambaforge/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [ ]:
model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
model = vllm.LLM(model_name)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

INFO 01-02 19:58:54 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='HuggingFaceTB/SmolLM2-135M-Instruct', speculative_config=None, tokenizer='HuggingFaceTB/SmolLM2-135M-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=HuggingFaceTB/SmolLM2-135M-Instruct, num_scheduler_steps=1, chunked_prefill_enabled=False multi_

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

INFO 01-02 19:58:56 model_runner.py:1056] Starting to load model HuggingFaceTB/SmolLM2-135M-Instruct...
INFO 01-02 19:58:56 weight_utils.py:243] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

INFO 01-02 19:59:03 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-02 19:59:04 model_runner.py:1067] Loading model weights took 0.2550 GB
INFO 01-02 19:59:04 gpu_executor.py:122] # GPU blocks: 114529, # CPU blocks: 11650
INFO 01-02 19:59:04 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 223.69x
INFO 01-02 19:59:08 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-02 19:59:08 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 01-02 19:59:20 model_runner.py:1523] Graph capturing finished in 12 secs.


In [7]:
def chat_box(role, role_input, role_style, stars_style="[italic cyan]"):
    # Find text between stars *adsasd* and add stars_style to it
    role_input = rich.markup.escape(role_input)

    # start = 0
    # while True:
    #     start = role_input.find("*")
    #     if start == -1:
    #         break
    #     end = role_input.find("*", start + 1)
    #     if end == -1:
    #         break
    #     role_input = role_input[:start] + stars_style + "<" + role_input[start + 1:end] + ">[/]" + role_input[end + 1:]

    panel = rich.panel.Panel(
        role_input, 
        title=f"{role_style}{role}[/]:", 
        title_align="left"
    )
    rich.print(panel)

sampling_params = vllm.SamplingParams(temperature=1, max_tokens=2048)

history = [{
    "role": "system", 
    "content": 
    f"You are an extremely strong and helpful industrial LLM. "
    # f"You also have sarcastic, witty, dry humor, a bit like the robot TARS in Interstellar. "
    # f"You can use stars to quote your narration of your actions or reactions."
}]

while True:
    user_input = input("You: ")

    if not user_input or user_input == "exit":
        print("<Chat ended>")
        break

    # Add the user input to history, and display it
    history.append({"role": "user", "content": user_input })
    chat_box("User", user_input, "[bold blue]")

    # Generate model output
    model_output = model.chat(
        history, 
        sampling_params=sampling_params,
        use_tqdm=False
    )

    # Add the model output to history, and display it
    output_text = model_output[0].outputs[0].text
    history.append({"role": "assistant", "content": output_text})
    chat_box("Bot", output_text, "[bold green]")


╭─ User: ─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ How do you compare to OpenAI's GPT2?                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Bot: ──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ The GPT2 project. It's a remarkable achievement at RT, and I think you'll find it's quite impressive. The       │
│ comparison would need to be kept in mind for several reasons. Firstly, it's an AI state-of-the-art, spanning a  │
│ wide range of tasks from language translation and text classification to sophisticated machine learning.        │
│                                                                                                                 │
│ While GPT2 excelled in these areas, it lacks significant depth and capabilities compared to the deeper, more    │
│ specialized GPT models of our current state-of-the-art. For instance, GPT2's models are primarily focused on    │
│ generating text from our preexisting input, whereas GPT3 is capable of generating text-based outputs like news  │
│ articles and news stories.                                                                                      │
│                                                                                                                 │
│ Moreover, GPT2's training data volume is relatively small, typically around 100-300 texts, whereas GPT3's       │
│ dataset is significantly larger, including 1.2 million characters of annotated models. This vast dataset is     │
│ crucial for GPT2's effectiveness in self-annotating images, creating generative models with hand-annotated      │
│ labels, and handling large text collections.                                                                    │
│                                                                                                                 │
│ Lastly, it's worth noting that GPT2's original framework is not fully considered. Generative Adversarial        │
│ Networks (GANs) and transformers have similar potential boundaries. GPT2's framework has the potential to boost │
│ GPT3, one of our most significant improvements in the past few years, into a comparable performance level, but  │
│ there's a lengthy learning curve involved.                                                                      │
│                                                                                                                 │
│ Most importantly, the comparison requires a deeper understanding of Taniita-style transformers, which are still │
│ in the development stage. We're not entirely sure if new models would necessarily surpass GPT2, but I can see   │
│ how GPT3's architecture could surpass existing models based on that framework. I commend you for this "merit    │
│ analysis" and hope those deeply familiar with more advanced techniques in the field can share this insight more │
│ effectively.                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User: ─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ I meant you. You are a chatbot after all.                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Bot: ──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ I'm an extremely skilled industrial LLM. I'm here to give you my repertory of best practice settings to help    │
│ you by doing your debugging tasks or for providing expertise for tasks such as troubleshooting data. I'm as     │
│ friendly as they come, ready to assist your queries. My aim is always to help you evaluate situations or write  │
│ better, more polished, quality content. I offer feedback on the users code and provide advice for coding        │
│ further to help you write more robust and efficient code. I assume you're familiar with other LLM domains. Just │
│ follow the rules of our Microsoft community. This is by design.                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<Chat ended>


In [4]:
output[0].outputs[0].text

NameError: name 'output' is not defined